# Radial Velocity Measurements of Exoplanets

This notebook demonstrates how to measure the mass of a planet using radial velocity data. 

# Installation

Run the following piece of code to install the radvel package. We will use a slightly modified version of radvel for running on Google Colab. 

In [ ]:
!git clone https://github.com/semaphoreP/radvel.git
!pip install -e ./radvel

## You need to restart your environment after running the previous commands

From the top menu: Runtime -> Restart Runtime. After you have restarted the runtime, proceed. 

# Setup

## Record RV measurements in a text file
Follow the format of [this example text file](https://github.com/semaphoreP/ASTRON314-414/blob/main/FinalProject/example_rvs.txt) to record your measurements from the literature. Each row contains a single epoch measurement: the time of observations (`time`), the radial velocity (`mnvel`), and the error on the radial velocity (`errvel`). You will definitely need more than the 3 points to measure the mass of the planet. 

To find radial velocity data, search the literature for the object you are interested in characterizing. Hint: look up the reference for the mass listed in the Exoplanet Archive (that paper likely contains these measurements or references the paper in which these measurements are published). 

When you have made your own text file, upload it to Colab.


## Import packages

In [86]:
import os
import copy
import pandas as pd
import numpy as np
import scipy.optimize as optimize
import radvel
import radvel.plot.orbit_plots as orbit_plots
import matplotlib.pylab as plt
%matplotlib inline


## Read in radial velocity data

Upload the data into the folder with this notebook, so that it can be read in. 

In [ ]:

# read file
data = pd.read_csv("example_rvs.txt", sep=' ')
# parse out the three columns into numpy arrays
t = np.array(data['time'])
vel = np.array(data['mnvel'])
errvel = data['errvel']

print(data)

# Setup the radial velocity model

The `radvel` package is good at fitting for most of the orbital parameters, but you need to specify the initial guess for the orbital period for the fit to work effectively (the `per1` field below). 

Note that `radvel` uses an orbital basis different from the one we disucssed in class. See Section 2.2 and Table 2 of [this paper](https://iopscience.iop.org/article/10.1088/1538-3873/aaaaa8/pdf) for details. In particular, you will care about `logk1`, which is the natural log of the RV semi-aplitude. 

In [90]:
# Create the orbital model. Change the orbital period to the orbital period of your planet!!
def initialize_model():
    time_base = 2456778
    params = radvel.Parameters(1,basis='per tc secosw sesinw logk')
    params['per1'] = radvel.Parameter(value=0.83753) # CHANGE THIS (unit is days)
    params['tc1'] = radvel.Parameter(value=2456778 + 1)
    params['secosw1'] = radvel.Parameter(value=0.01)
    params['sesinw1'] = radvel.Parameter(value=0.01)
    params['logk1'] = radvel.Parameter(value=1)
    mod = radvel.RVModel(params, time_base=time_base)
    return mod

## Determine which values to fit

For simplicity, we will fix the period, so we are not fitting for it. We also will fix $\sqrt{e}\sin(\omega)$ and $\sqrt{e}\cos(\omega)$ to nearly 0, which results in a nearly circular orbit. If your planet is known to be eccentric, you should let these parameters vary instead. We will obviously vary the RV semi-amplitude so we can measure the mass. 

In [ ]:
mod = initialize_model()
like = radvel.likelihood.RVLikelihood(mod, t, vel, errvel)
like.params['gamma'] = radvel.Parameter(value=0.1)
like.params['jit'] = radvel.Parameter(value=1.0)

like.params['secosw1'].vary = False
like.params['sesinw1'].vary = False
like.params['per1'].vary = False
like.params['tc1'].vary = True
print(like)

# Perform a simple optimization fit

This will give us some maximum-likelihood values for the RV semi-amplitude. The output that gets printed from this cell lists all the parameters for the model, and tells you which were varied in the optimization. 

In [ ]:
post = radvel.posterior.Posterior(like)

res  = optimize.minimize(
    post.neglogprob_array,     # objective function is negative log likelihood
    post.get_vary_params(),    # initial variable parameters
    method='Powell',           # Nelder-Mead also works
    )
             # plot best fit model
print(post)

# Plot the orbital fit

We will plot both the regular fit in time, and the phase-folded fit. The plot will also list the best-fit parameter for the RV semi-amplitude. 

In [ ]:
post.params['dvdt'] = radvel.Parameter(value=0)
post.params['curv'] = radvel.Parameter(value=0)

RVPlot = orbit_plots.MultipanelPlot(post)

RVPlot.plot_multipanel()